This notebook plots SEDs and the corresponding photometry.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import medfilt
from matplotlib.ticker import MaxNLocator
from moviepy.editor import ImageSequenceClip
from glob import glob

In [ ]:
# load the filters
filters = {}
for band in "ugrizy":
    x, y = np.genfromtxt(f"data/{band}_bandpass.dat", unpack=True)
    x *= 10
    y *= x
    y /= np.trapz(y, x)
    filters[band] = np.array([x, y])

# calculate the effective wavelengths
filter_wavelen = np.array([np.trapz(x * y, x) for (x, y) in filters.values()])

In [ ]:
def plot_filters(ax):
    # get the span and orientation of the plot
    ymin, ymax = ax.get_ylim()
    span = np.abs(ymax - ymin)
    scale = 1 if ymax > ymin else -1

    # loop through and plot the filters
    for (x, y) in filters.values():
        y_scaled = y / y.max() * 0.5 * span
        plt.fill_between(x, ymin, ymin + scale * y_scaled, color="silver", alpha=0.4)

In [ ]:
def load_sed(file, kernel=7):
    # load the SED from the file
    sed = np.genfromtxt(f"data/{file}.sed").T

    # cut off the SED above 12,000 Angstroms
    sed = sed[:, :np.abs(sed[0] - 12_000).argmin()]

    # rescale the SED so that magnitudes are realistic
    sed[1] *= 1e-16

    # smooth SED using the kernel
    sed[1] = medfilt(sed[1], kernel)

    # append the SED in AB magnitude
    with np.errstate(divide='ignore'):
        sed = np.vstack((sed, -2.5 * np.log10(9.1986 * sed[0]**2 * sed[1])))
    sed[2] = np.clip(sed[2], None, 30)

    return sed

In [ ]:
def redshift_sed(sed, z):
    # stretch the wavelength grid
    redshifted_wavelen = (1 + z) * sed[0]

    # scale down the flux density
    redshifted_flambda = sed[1] / (1 + z)

    # interpolate on the original grid
    redshifted_flambda = np.interp(sed[0], redshifted_wavelen, redshifted_flambda)

    # recalculate magnitudes
    with np.errstate(divide='ignore'):
        redshifted_mags = -2.5 * np.log10(9.1986 * sed[0]**2 * redshifted_flambda)
    redshifted_mags = np.clip(redshifted_mags, None, 30)
    
    return np.array([sed[0], redshifted_flambda, redshifted_mags])

In [ ]:
def calculate_photometry(sed):
    fluxes = []
    for ((x, y), w) in zip(filters.values(), filter_wavelen):
        # interpolate the filter onto the SED grid
        y = np.interp(sed[0], x, y, left=0, right=0)

        # calculate the integrated flux
        fluxes.append(np.trapz(sed[1] * y, sed[0]))

    return np.array(fluxes)

# calculate the zero point flux for each band
flat_sed = np.linspace(100, 12_000, 1000)
flat_sed = np.array([flat_sed, 1 / (9.1986 * flat_sed**2)])
zero_points = calculate_photometry(flat_sed)

In [ ]:
def plot_photometry(ax, sed, mags=False):
    # calculate photometry
    photometry = calculate_photometry(sed)

    # calculate magnitudes?
    if mags:
        photometry = -2.5 * np.log10(photometry / zero_points)

    # plot the photometry
    ax.scatter(
        filter_wavelen,
        photometry,
        marker="*",
        s=100,
        lw=0.65,
        c="C1",
        edgecolors='k',
        zorder=10, 
    )

In [ ]:
El = load_sed("El_B2004a")
Sb = load_sed("SB2_B2004a")

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax.set(
    xlim=(2_500, 11_000),
    ylim=(0, 1e-15),
    xlabel="Wavelength ($\AA$)",
    ylabel="Flux density",
)
ax.ticklabel_format(axis="y", useMathText=True)

# plot the original SED
ax.plot(Sb[0], Sb[1])

# label the redshift
ax.text(0.97, 0.97, "z=0", ha="right", va="top", transform=ax.transAxes)

plt.tight_layout()
fig.savefig("figures/restframe_sed.png", dpi=500)

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax.set(
    xlim=(2_500, 11_000),
    ylim=(0, 1e-15),
    xlabel="Wavelength ($\AA$)",
    ylabel="Flux density",
)
ax.ticklabel_format(axis="y", useMathText=True)

# plot the original SED
ax.plot(Sb[0], Sb[1])

# plot the redshifted SED
redshifted_sed = redshift_sed(Sb, 0.2)
ax.plot(redshifted_sed[0], redshifted_sed[1], c="C3")

# draw arrows indicating the redshift
ax.annotate(
    "",
    xy=(4400, 0.5e-15),
    xytext=(3800, 0.5e-15),
    arrowprops=dict(arrowstyle="->", color="k"),
)
ax.annotate(
    "",
    xy=(5975, 0.5e-15),
    xytext=(5075, 0.5e-15),
    arrowprops=dict(arrowstyle="->", color="k"),
)
ax.annotate(
    "",
    xy=(7850, 0.5e-15),
    xytext=(6625, 0.5e-15),
    arrowprops=dict(arrowstyle="->", color="k"),
)

# label the redshift
ax.text(0.97, 0.97, "z=0.25", ha="right", va="top", transform=ax.transAxes)

plt.tight_layout()
fig.savefig("figures/redshifted_sed.png", dpi=500)

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax.set(
    xlim=(2_500, 11_000),
    ylim=(0, 1e-15),
    xlabel="Wavelength ($\AA$)",
    ylabel="Flux density",
)
ax.ticklabel_format(axis="y", useMathText=True)

# plot the original SED
ax.plot(Sb[0], Sb[1])

# label the redshift
ax.text(0.97, 0.97, "z=0", ha="right", va="top", transform=ax.transAxes)

# plot filters and photometry
plot_filters(ax)
plot_photometry(ax, Sb)

plt.tight_layout()
fig.savefig("figures/restframe_photometry.png", dpi=500)

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax.set(
    xlim=(2_500, 11_000),
    ylim=(19.5, 15.75),
    xlabel="Wavelength ($\AA$)",
    ylabel="Magnitude",
)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

# plot the original SED
ax.plot(Sb[0], Sb[2])

# label the redshift
ax.text(0.97, 0.97, "z=0", ha="right", va="top", transform=ax.transAxes)

# plot filters and photometry
plot_filters(ax)
plot_photometry(ax, Sb, mags=True)

plt.tight_layout()

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)

# directory where individual frames are stored
sed_redshift_dir = "figures/sed_redshift"

# plot the redshifted SED
for i, redshift in enumerate(np.arange(0, 1.8, 0.05)):
    ax.clear()
    
    ax.set(
        xlim=(2_500, 11_000),
        ylim=(0, 1.2e-16),
        xlabel="Wavelength ($\AA$)",
        ylabel="Flux density",
    )
    ax.ticklabel_format(axis="y", useMathText=True)

    redshifted_sed = redshift_sed(El, redshift)
    ax.plot(redshifted_sed[0], redshifted_sed[1], c="C0")

    # label the redshift
    ax.text(0.97, 0.97, f"z={redshift:.2f}", ha="right", va="top", transform=ax.transAxes)

    # plot filters and photometry
    plot_filters(ax)
    plot_photometry(ax, redshifted_sed)

    # only set the tight layout on the first iteration
    if i == 0:
        plt.tight_layout()

    # save the frame
    fig.savefig(f"{sed_redshift_dir}/step{i}.png", dpi=500)

# get the list of frames
file_list = [f"{sed_redshift_dir}/step{i}.png" for i in range(len(glob(f"{sed_redshift_dir}/*")))]

# combine the frames into a video
sed_redshift_clip = ImageSequenceClip(file_list, fps=4)
sed_redshift_clip.write_gif("figures/sed_redshift.gif")
